In [1]:
import requests
import json

In [2]:
client_id = ""
client_secret = ""
api_key = ""

with open('config.json', 'r') as file:
    data = json.load(file)

client_id = data.get('client_id', '')
client_secret = data.get('client_secret', '')
api_key = data.get('api_key', '')

# Spotify
### Methods to get access token, genre from artist name, and track URI

In [9]:
def get_spotify_access_token(client_id, client_secret):
    url = "https://accounts.spotify.com/api/token"
    data = {"grant_type": "client_credentials"}
    response = requests.post(url, data=data, auth=(client_id, client_secret))
    token_data = response.json()
    return token_data["access_token"]
    
def get_genre_for_artist(artist_name, access_token):
    url = f"https://api.spotify.com/v1/search?q={artist_name}&type=artist"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    data = response.json()
    # print(data)
    if 'artists' in data and 'items' in data['artists'] and data['artists']['items']:
        artist = data['artists']['items'][0]
        if 'genres' in artist:
            genres = artist['genres']
            return genres
    return []

def get_track_uri(track_name, artist_name, access_token):
    search_query = f"track:{track_name} artist:{artist_name}"
    url = f"https://api.spotify.com/v1/search?q={search_query}&type=track"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    data = response.json()

    if 'tracks' in data and 'items' in data['tracks'] and data['tracks']['items']:
        track_uri = data['tracks']['items'][0]['uri']
        return track_uri

    return None

def get_audio_features(track_uris, access_token):
    url = "https://api.spotify.com/v1/audio-features"
    headers = {
        "Authorization": f"Bearer {access_token}",
    }
    params = {"ids": ",".join(track_uris)}
    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    if 'audio_features' in data:
        return data['audio_features']

    return []

In [10]:
artist_name = "The Isley Brothers"
track_name = "Here We Go Again, Pts. 1 & 2"

access_token = get_spotify_access_token(client_id, client_secret)

genres = get_genre_for_artist(artist_name, access_token)
track_uri = get_track_uri(track_name, artist_name, access_token)
audio_features = get_audio_features([track_uri.split(":")[-1]], access_token)

print(f"Genres for {artist_name}: {', '.join(genres)}")
print(f"Track URI for {track_name}: {track_uri}")

for track in audio_features:
    if not track:
        continue

    print(f"Track URI: {track['uri']}")
    print(f"Acousticness: {track['acousticness']}")
    print(f"Danceability: {track['danceability']}")
    print(f"Energy: {track['energy']}")
    print(f"Valence: {track['valence']}")

Genres for The Isley Brothers: classic soul, disco, funk, motown, quiet storm, soul
Track URI for Here We Go Again, Pts. 1 & 2: spotify:track:0z5qnWq8cOqHJoegSKGwjt
Track URI: spotify:track:0z5qnWq8cOqHJoegSKGwjt
Acousticness: 0.291
Danceability: 0.679
Energy: 0.722
Valence: 0.905


# Last.fm
### Methods to get genre from artist name

In [11]:
def get_genre_for_artist(track_name, artist_name):
    url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&track={track_name}&artist={artist_name}&api_key={api_key}&format=json"
    response = requests.get(url)
    data = response.json()
    # print(data)
    if 'artist' in data and 'tags' in data['artist']:
        genres = [tag['name'] for tag in data['artist']['tags']['tag']]
        return genres
    else:
        return []
    
genres = get_genre_for_artist(track_name, artist_name)
print(f"Genres for the track '{track_name}' by {artist_name}: {', '.join(genres)}")


Genres for the track 'Here We Go Again, Pts. 1 & 2' by The Isley Brothers: soul, funk, motown, rhythm and blues, rnb
